In [ ]:
import os
import numpy as np

from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.stem import PorterStemmer

#preprocessing
       
files_name=natsorted(os.listdir('files'))
document_of_terms=[]
for files in files_name:
    with open(f'files/{files}','r') as f:
        document=f.read()
        
         
    tokenized_documents=word_tokenize(document)
    terms=[]
    for word in tokenized_documents:
            terms.append(word)
    document_of_terms.append(terms)

print('                                               *****    Documents in  Terms   *****                                                                        \n    '
    ,  document_of_terms,'\n')                        

#stemming 
stemmer=PorterStemmer()
document_of_stemms=[]
for terms in document_of_terms:
     for word in terms :
         stemmed_terms=[stemmer.stem(word)]
         document_of_stemms.append(stemmed_terms)

print('                                               *****     Stemmed Terms   *****                                                                        \n    '
    ,  document_of_stemms,'\n')        

#positional_index

document_number = 0
positional_index = {}

print() 
print("                                               *****     positions    *****                                       \n   ")
for document in document_of_terms:
    for positional, term in enumerate(document):
        if term in positional_index:
            positional_index[term][0] = positional_index[term][0] + 1

            if document_number in positional_index[term][1]:
                positional_index[term][1][document_number].append(positional)
            else:
                positional_index[term][1][document_number] = [positional]

        else:
            positional_index[term] = []
            positional_index[term].append(1)
            positional_index[term].append({})
            positional_index[term][1][document_number] = [positional]

    document_number += 1
    

print(positional_index,'\n')


#query_preprocessing

query=input('put query here:')

final_list=[[]for i in range (10)]
print('                                    ***** all response list *****                                                     ')
for word in query.split():    
    if word in positional_index.keys():
        for key in positional_index[word][1].keys():
            
            if final_list[key]!=[]:
                
                 if final_list[key][-1] == positional_index[word][1][key][0]-1:
                        final_list[key].append(positional_index[word][1][key][0])
        
            else:
                 final_list[key].append(positional_index[word][1][key][0])
            
print(final_list,'\n')

print('                                  *****response list to query*****                                              ')
for position,list in enumerate(final_list,start=1):
    if len(list)==len(query.split()):
      print(position,list)     



#find tf and wtf
import pandas as pd
import math


all_words=[]
for doc in document_of_terms:
    for word in doc:
        all_words.append(word)
 #tf       
def get_term_freq(doc):
    words_found=dict.fromkeys(all_words,0)
    for word in doc:
        words_found[word]+=1
    return words_found

term_freq=pd.DataFrame(get_term_freq(document_of_terms[0]).values(),index=get_term_freq(document_of_terms[0]).keys())              
print('                                  *****term freq for each  term in doc*****                                              ')
print()        
for i in range(1,len(document_of_terms)):
    term_freq[i]=get_term_freq(document_of_terms[i]).values()
    
term_freq.columns=['doc'+str(i) for i in range (1,11)]    
print(term_freq)     

#wtf

def get_weighted_term_freq(x):
    if x>0:
      return math.log(x)+1
    return 0
     
print('                                  ***** weighted term freq for each  term in doc*****                                              ')
print()        
for i in range(1,len(document_of_terms)+1):
    term_freq['doc'+str(i)]= term_freq['doc'+str(i)].apply(get_weighted_term_freq) 
    
print(term_freq,'\n')
print()
 
#idf and tf*idf
 
tfd=pd.DataFrame(columns=['df','idf'])

print('                                  *****        IDF          *****                                              ')
for i in range(len(term_freq)):
    
    frequancy=term_freq.iloc[i].values.sum()
    
    tfd.loc[i,'df']=frequancy
    tfd.loc[i,'idf']=math.log10(10/(float(frequancy)))
    
tfd.index=term_freq.index
    
print(tfd)

# tf*idf

tf_idf=term_freq.multiply(tfd['idf'],axis=0)

print('                                  *****         TF*IDF             *****                                              ')
print()      
print(tf_idf)


# doc length

def get_doc_len(col):
    return np.sqrt(tf_idf[col].apply(lambda x:x**2).sum())

doc_len=pd.DataFrame()
for col in tf_idf.columns:
    doc_len.loc['length', col+'__length']=get_doc_len(col)   
print('                                  *****         Doc Length             *****                                              ') 
print(doc_len,'\n')    



# normalization
norm_tf_idf= tf_idf.divide(doc_len.values[0],axis=1)
print('                                  *****        Normalized TF*IDF           *****                                              ')
print()
print(norm_tf_idf)



# positional index median


In [ ]:
def postfix(infix_tokens):
 
 #precendence initialization
 precedence = {}
 precedence['NOT'] = 3
 precedence['AND'] = 2
 precedence['OR'] = 1
 precedence['('] = 0
 precedence[')'] = 0
 output = []
 operator_stack = []
 
 #creating postfix expression
 for token in infix_tokens:
    if (token == '('):
        operator_stack.append(token)
    elif (token == ')'):
        operator = operator_stack.pop()
        while (operator != '('):
                 output.append(operator)
                 operator = operator_stack.pop()
 
    elif (token in precedence):
        if (operator_stack):
             current_operator = operator_stack[-1]
             while (operator_stack and precedence[current_operator] > precedence[token]):
                output.append(operator_stack.pop())
                    if (operator_stack):
                        current_operator = operator_stack[-1]
                        operator_stack.append(token)
                    else:
                            output.append(token.lower())
 
 #while staack is not empty appending
while (operator_stack):
      output.append(operator_stack.pop())
return output

In [ ]:
def process_query(q,dictionary_inverted):
  q = q.replace('(', '( ')
  q = q.replace(')', ' )')
  q = q.split('' )
  query = []
for i in q:
    query.append(ps.stem(i))
        for i in range(0,len(query)):
             if ( query[i]== 'and' or query[i]== 'or' or query[i]== 'not'):
                 query[i] = query[i].upper()
                 results_stack = []
                 postfix_queue = postfix(query)
         
         #evaluating postfix query expression
                     for i in postfix_queue:
                          i = i.replace('(', '')
                           i = i.replace(')', '')
                          i = i.lower()
                          i = dictionary_inverted.get(i)
                             results_stack.append(i)
             
             elif (i=='AND'):
                 a = results_stack.pop()
                 b = results_stack.pop()
                 results_stack.append(AND_op(a,b))
             elif (i=='OR'):
                a = results_stack.pop()
                b = results_stack.pop()
                results_stack.append(OR_op(a,b))
             elif (i == 'NOT'):
                a = results_stack.pop()
                print(a)
                results_stack.append(NOT_op(a))
 
 return results_stack.pop()

In [ ]:
#AND two posting lists
def AND_op(word1,word2):
    if ((word1) and (word2)):
       return list(word1).intersection(word2)
    else:
       return list()
 
#OR two posting lists
def OR_op(word1,word2):
 return list(word1).union(word2)
 
#NOT two posting lists
def NOT_op(a):
   tot_docs = list(range(0,56))
 return list(tot_docs).symmetric_difference(a)

# sol deliverd

In [ ]:
# Handling boolean operators

import os
import numpy as np

from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.stem import PorterStemmer as porter_stemmer


files_name=natsorted(os.listdir('files'))
document_of_terms=[]
for files in files_name:
    with open(f'files/{files}','r') as f:
        document=f.read()
        
boolean_operators = ['and', 'or', 'not']
boolean_operator = None

for op in boolean_operators:
    if op in document_of_terms:
        boolean_operator = op
        break

matched_documents = None


query=input('put query here:')
# Split the query based on the boolean operator
query_parts = query.lower().split(boolean_operator)
# Tokenization and stemming for each part
query_parts_terms = [
    [porter_stemmer.stem(term) for term in document_of_terms(term)] for part in query_parts
]
print(query_parts_terms)

if boolean_operator:
    # Find matched documents for each part
    matched_documents_parts = []
    for part_terms in query_parts_terms:
        matched_documents_part = set(range(1, len(document_lengths) + 1))  # Initialize with all documents
        for term in part_terms:
            if term in positional_index:
                matched_documents_part &= set(positional_index[term]['docs'].keys()) # type: ignore
        matched_documents_parts.append(matched_documents_part)
    print(matched_documents_parts)
    # Apply boolean operator
    if boolean_operator == 'and':
        matched_documents = set.intersection(*matched_documents_parts)
    elif boolean_operator == 'or':
        matched_documents = set.union(*matched_documents_parts)
    elif boolean_operator == 'not':
        matched_documents = set.difference(set(range(1, len(document_lengths) + 1)), *matched_documents_parts)
    
else:
    # No boolean operator, find matched documents without considering boolean logic
    matched_documents = set(range(1, len(document_lengths) + 1))
    for term in document_of_terms:
        if term in positional_index:
            matched_documents &= set(positional_index[term]['docs'].keys()) # type: ignore

print("Matched Documents:", matched_documents)


# Get distinct terms in the query
document_of_terms = [item for sublist in query_parts_terms for item in sublist]
original_document_of_terms = document_of_terms
document_of_terms = list(set(document_of_terms))

# bart of bool fun


In [ ]:
def intersection(list1, list2):
    return list(set(list1).intersection(list2))
def union(list1, list2):
    return list(set(list1).union(list2))
def notin(list1, list2):
    return [filter(lambda x: x not in list1, sublist) for sublist in list2]

intersection(positional_index['fools'], intersection(positional_index['fear'], positional_index['angels']))